In [1]:
import numpy as np
import os
import sys
from astropy.table import QTable, Table, Column, vstack
import multiprocessing
import time

In [95]:
def generate_one_line(n_inject, trail_length, mag, beta, butler, ref, input_coll, dimensions, source_type):
    injection_catalog = Table(
        names=('injection_id', 'ra', 'dec', 'source_type', 'trail_length', 'mag', 'beta', 'visit', 'integrated_mag', 'PSF_mag',
               'physical_filter'),
        dtype=('int64', 'float64', 'float64', 'str', 'float64', 'float64', 'float64', 'int64', 'float64', 'float64', 'str'))
    injection_catalog.add_index('injection_id')
    raw = butler.get(
        source_type + ".wcs",
        dataId=ref.dataId,
        collections=input_coll,
    )
    info = butler.get(
        source_type + ".visitInfo",
        dataId=ref.dataId,
        collections=input_coll,
    )
    filter_name = butler.get(
        source_type + ".filter",
        dataId=ref.dataId,
        collections=input_coll,
    )
    
    #parameters to use from http://arxiv.org/pdf/1711.10621
    fwhm = {"u": 0.92, "g": 0.87, "r": 0.83, "i": 0.80, "z": 0.78, "y": 0.76}
    # Taken from https://smtn-002.lsst.io/#source-footprint-n-eff
    m5 = {"u": 23.7, "g": 24.97, "r": 24.52, "i": 24.13, "z": 23.56, "y": 22.55}
    # Taken from https://smtn-002.lsst.io/#calculating-m5
    psf_depth = m5[filter_name.bandLabel]
    pixelScale = raw.getPixelScale().asArcseconds()
    theta_p = fwhm[filter_name.bandLabel]*pixelScale
    a = 0.67
    b = 1.16
    
    # calculating image bounds in skycoordinates
    start = raw.pixelToSky(0, 0)
    end = raw.pixelToSky(dimensions[0], dimensions[1])
    min_ra = start.getRa()
    min_dec = start.getDec()
    max_ra = end.getRa()
    max_dec = end.getDec()
    diff_ra = max_ra - min_ra
    diff_dec = max_dec - min_dec
    min_ra = min_ra + 0.02 * diff_ra
    max_ra = max_ra - 0.02 * diff_ra
    min_dec = min_dec + 0.02 * diff_dec
    max_dec = max_dec - 0.02 * diff_dec
    
    for k in range(n_inject):
        ra_pos = np.random.uniform(low=min_ra.asDegrees(), high=max_ra.asDegrees())
        dec_pos = np.random.uniform(low=min_dec.asDegrees(), high=max_dec.asDegrees())
        inject_length = np.random.uniform(low=trail_length[0], high=trail_length[1])
        x = inject_length / (24 * theta_p)
        if mag[1] == 0:
            # calculating the upper limit magnitude based on the trail length and the maximum detectable limit
            # Taken from Jones et al. 2017: http://arxiv.org/pdf/1711.10621
            upper_limit_mag = psf_depth - 1.25 * np.log10(1 + (a * x ** 2) / (1 + b * x))
        else:
            # user defined magnitude limits
            upper_limit_mag = mag[1]
        # rolling dice for the magnitude then calculating the surface brightness
        magnitude = np.random.uniform(low=mag[0], high=upper_limit_mag)
        surface_brightness = magnitude + 2.5 * np.log10(inject_length)
        psf_magnitude = magnitude + 1.25 * np.log10(1 + (a * x ** 2) / (1 + b * x))
        # rolling dice for the surface brightness then calculating the magnitude
        # surface_brightness = np.random.uniform(low=mag[0], high=mag[1])
        # magnitude = surface_brightness - 2.5 * np.log10(inject_length)
        angle = np.random.uniform(low=beta[0], high=beta[1])
        visitid = info.id
        injection_catalog.add_row([k, ra_pos, dec_pos, "Trail", inject_length, surface_brightness, angle, visitid,
                                   magnitude, psf_magnitude, filter_name.bandLabel])
    return injection_catalog


def generate_catalog(repo, input_coll, n_inject, trail_length, mag, beta, where="", verbose=True,
                     multiprocess_size=None):
    """
    Create a catalog of trails to be injected in the input collection for the source injection. The catalog is saved in the
    astropy table format. The catalog is created by randomly selecting a position in the input collection and then
    randomly selecting the trail length, magnitude and angle of the trail. The catalog will have a visit id for each trail
    that is the same as the visit id of the calexp that the trail is injected into.

    :param repo:
    :param input_coll:
    :param n_inject:
    :param trail_length:
    :param mag:
    :param beta:
    :param where:
    :param verbose:
    :return:
    """
    from lsst.daf.butler import Butler
    butler = Butler(repo)
    registry = butler.registry
    source_type = "calexp"
    if where == "":
        query = set(registry.queryDatasets(source_type, collections=input_coll, instrument='HSC', findFirst=True))
    else:
        query = set(registry.queryDatasets(source_type, collections=input_coll, instrument='HSC', where=where, findFirst=True))
    length = len(list(query))
    dimensions = butler.get(
        source_type + ".dimensions",
        dataId=list(query)[0].dataId,
        collections=input_coll,
    )
    parameters = [(n_inject, trail_length, mag, beta, butler, ref,
                   input_coll, dimensions, source_type,) for ref in query]
    if verbose:
        print("Number of visits found: ", length)
    if multiprocess_size is None:
        multiprocess_size = max(1, min(os.cpu_count() - 1, len(parameters)))
    if multiprocess_size > 1:
        with multiprocessing.Pool(multiprocess_size) as pool:
            injection_catalog = pool.starmap(generate_one_line, parameters)
    else:
        injection_catalog = [None] * len(parameters)
        for i in range(len(parameters)):
            injection_catalog[i] = generate_one_line(*parameters[i])
            if verbose:
                print("\r", i + 1, "/", length, end="", flush=True)
        if verbose:
            print("")
    output_catalog = vstack(injection_catalog, join_type='exact')
    output_catalog["injection_id"] = np.arange(len(output_catalog))
    return output_catalog

In [98]:
repo = '/epyc/ssd/users/kmrakovc/DATA/rc2_subset/SMALL_HSC/'
input_collection = "u/kmrakovc/RC2_subset/run_1"
number = 20
trail_length = [6, 60]
magnitude = [20, 25]
beta = [0, 180]
start_time = time.time()
catalog = generate_catalog(repo, input_collection, number, trail_length, magnitude,
                               beta, verbose=True, multiprocess_size=1)
print ("Total time:", time.time()-start_time)

Number of visits found:  240
 240 / 240
Total time: 44.00261354446411


In [99]:
catalog

injection_id,ra,dec,source_type,trail_length,mag,beta,visit,integrated_mag,PSF_mag,physical_filter
int64,float64,float64,str5,float64,float64,float64,int64,float64,float64,str1
0,150.0903850154769,2.253038059361637,Trail,42.14474694161813,26.294435247072915,165.3703644868937,358,22.23257662093088,23.390912937787633,y
1,150.19160728993705,2.256437169520551,Trail,9.5850392759723,25.92929667817366,44.96802220072273,358,23.475311936180947,23.952842620461013,y
2,150.1137503953228,2.2085346976613027,Trail,32.028689046915865,24.37160570590102,111.60426615676577,358,20.607757798153578,21.6284858109673,y
3,150.12328458081,2.2185065394997374,Trail,45.62942252757856,26.675691439927277,39.60364775930339,358,22.527579008663665,23.72629996463411,y
4,150.20799107373696,2.2432249972601714,Trail,26.178908318873198,26.495561706083876,47.28312654159543,358,22.9506828756592,23.872599011309543,y
5,150.17671446910654,2.260460435758058,Trail,15.83357544626311,23.844151627134618,109.15556405712314,358,20.8452041376339,21.53253819280962,y
6,150.18572429604095,2.2018938957962053,Trail,7.403322786766402,26.66530330019261,19.441887951460842,358,24.49173658732896,24.874425812588115,y
7,150.22650041024747,2.193539296916542,Trail,16.18921777331552,25.99037518360516,13.802804917093269,358,22.967310520792093,23.6645690044071,y
8,150.2010041029058,2.2445991900325937,Trail,42.57658510123765,24.649145934228056,104.8085565885114,358,20.57621887056828,21.739724868486697,y


In [4]:
from lsst.daf.butler import Butler
repo = '/epyc/ssd/users/kmrakovc/DATA/rc2_subset/SMALL_HSC/'
input_coll = "u/kmrakovc/RC2_subset/run_1"
butler = Butler(repo)
registry = butler.registry
source_type = "calexp"
query = registry.queryDatasets(source_type, collections=input_coll, instrument='HSC')
ref = list(query)[0]
raw = butler.get(
        source_type,
        dataId=ref.dataId,
        collections=input_coll,
    )

In [8]:
dir(raw)

['Factory',
 '__add__',
 '__class__',
 '__deepcopy__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__mul__',
 '__ne__',
 '__new__',
 '__radd__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rmul__',
 '__rsub__',
 '__rtruediv__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__sub__',
 '__subclasshook__',
 '__truediv__',
 '_get',
 '_set',
 'apCorrMap',
 'clone',
 'containsSkyCoords',
 'convertD',
 'convertF',
 'convex_polygon',
 'detector',
 'dtype',
 'filter',
 'getBBox',
 'getConvexPolygon',
 'getCutout',
 'getDetector',
 'getDimensions',
 'getFilter',
 'getHeight',
 'getImage',
 'getInfo',
 'getMask',
 'getMaskedImage',
 'getMetadata',
 'getPhotoCalib',
 'getPsf',
 'getVariance',
 'getWcs',
 'getWidth',
 'getX0',
 'getXY0',
 'getY0',
 'hasPsf',
 'hasWcs',
 'height',
 'image',
 'info

In [43]:
dir(raw.getPsf())

['COPY',
 'INTERNAL',
 'ImageOwnerEnum',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'clone',
 'computeApertureFlux',
 'computeBBox',
 'computeImage',
 'computeImageBBox',
 'computeKernelBBox',
 'computeKernelImage',
 'computePeak',
 'computeShape',
 'getAverageColor',
 'getAveragePosition',
 'getCacheCapacity',
 'getKernel',
 'getLocalKernel',
 'isPersistable',
 'readFits',
 'recenterKernelImage',
 'resized',
 'setCacheCapacity',
 'write',
 'writeFits']

In [72]:
psf = raw.getPsf()
wcs = raw.getWcs()
sigma = psf.computeShape(wcs.skyToPixel(wcs.pixelToSky(100,100))).getDeterminantRadius()
pixelScale = raw.getWcs().getPixelScale().asArcseconds()

In [77]:
dir (psf)

['COPY',
 'INTERNAL',
 'ImageOwnerEnum',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'clone',
 'computeApertureFlux',
 'computeBBox',
 'computeImage',
 'computeImageBBox',
 'computeKernelBBox',
 'computeKernelImage',
 'computePeak',
 'computeShape',
 'getAverageColor',
 'getAveragePosition',
 'getCacheCapacity',
 'getKernel',
 'getLocalKernel',
 'isPersistable',
 'readFits',
 'recenterKernelImage',
 'resized',
 'setCacheCapacity',
 'write',
 'writeFits']

In [71]:
psf.computeShape(wcs.skyToPixel(wcs.pixelToSky(100,100)))

Quadrupole(ixx=7.134939548886614, iyy=6.3665688626143275, ixy=0.19049972941591878)

In [70]:
dir(psf.computeShape(wcs.skyToPixel(wcs.pixelToSky(100,100))))

['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__nq__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'assign',
 'clone',
 'computeDimensions',
 'convolve',
 'getArea',
 'getDeterminantRadius',
 'getIxx',
 'getIxy',
 'getIyy',
 'getName',
 'getParameterVector',
 'getTraceRadius',
 'grow',
 'normalize',
 'scale',
 'setIxx',
 'setIxy',
 'setIyy',
 'setParameterVector',
 'transform',
 'transformInPlace']

In [60]:
wcs.skyToPixel(wcs.pixelToSky(0,0))

Point2D(-2.8564134026964894e-06, -4.3138662476849277e-05)

In [48]:
dir(raw.wcs)

['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'copyAtShiftedPixelOrigin',
 'getCdMatrix',
 'getFitsMetadata',
 'getFrameDict',
 'getPixelOrigin',
 'getPixelScale',
 'getRelativeRotationToWcs',
 'getSkyOrigin',
 'getTanWcs',
 'getTransform',
 'isFits',
 'isFlipped',
 'isPersistable',
 'linearizePixelToSky',
 'linearizeSkyToPixel',
 'pixelToSky',
 'pixelToSkyArray',
 'readFits',
 'readString',
 'skyToPixel',
 'skyToPixelArray',
 'writeFits',
 'writeString']

In [40]:
raw.info.KEY_PSF

'PSF'

In [25]:
dir(raw.photoCalib)

['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '_isConstant',
 'calibrateCatalog',
 'calibrateImage',
 'computeScaledCalibration',
 'computeScalingTo',
 'getCalibrationErr',
 'getCalibrationMean',
 'getInstFluxAtZeroMagnitude',
 'getLocalCalibration',
 'getLocalCalibrationArray',
 'instFluxToMagnitude',
 'instFluxToMagnitudeArray',
 'instFluxToNanojansky',
 'isPersistable',
 'magnitudeToInstFlux',
 'magnitudeToInstFluxArray',
 'readFits',
 'writeFits']

In [26]:
raw.photoCalib.calibrateCatalog

<bound method PyCapsule.calibrateCatalog of PhotoCalib(spatially constant with mean: 0.768817 error: 0.00079694)>

In [32]:
dir(raw.psf)

['COPY',
 'INTERNAL',
 'ImageOwnerEnum',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'clone',
 'computeApertureFlux',
 'computeBBox',
 'computeImage',
 'computeImageBBox',
 'computeKernelBBox',
 'computeKernelImage',
 'computePeak',
 'computeShape',
 'getAverageColor',
 'getAveragePosition',
 'getCacheCapacity',
 'getKernel',
 'getLocalKernel',
 'isPersistable',
 'readFits',
 'recenterKernelImage',
 'resized',
 'setCacheCapacity',
 'write',
 'writeFits']

In [29]:
raw.psf

In [20]:
dir(raw.visitInfo)

['__class__',
 '__deepcopy__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'boresightAirmass',
 'boresightAzAlt',
 'boresightHourAngle',
 'boresightParAngle',
 'boresightRaDec',
 'boresightRotAngle',
 'copyWith',
 'darkTime',
 'date',
 'era',
 'exposureTime',
 'focusZ',
 'getBoresightAirmass',
 'getBoresightAzAlt',
 'getBoresightHourAngle',
 'getBoresightParAngle',
 'getBoresightRaDec',
 'getBoresightRotAngle',
 'getDarkTime',
 'getDate',
 'getEra',
 'getExposureTime',
 'getFocusZ',
 'getHasSimulatedContent',
 'getId',
 'getInstrumentLabel',
 'getLocalEra',
 'getObject',
 'getObservationReason',
 'getObservationType',
 'getObservatory',
 'getRotType',
 'getScienceProgram',
 'getUt1',
 'getW

In [22]:
raw.visitInfo.focusZ

3.7

In [7]:
raw.visitInfo.exposureTime

240.0

In [89]:
np.arange(len(catalog))

array([   0,    1,    2, ..., 4797, 4798, 4799])

In [87]:
catalog.info.info_summary_stats()

TypeError: data_info_factory.<locals>.func() missing 1 required positional argument: 'dat'